In [3]:
# !pip install selenium

## Necessary Imports

In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from PIL import Image
import os

import pandas as pd
import re
import requests

from time import sleep
import time

## Scraping

In [1]:
usernames = #list of usernames
passwords = #list of passwords
months = #list of months
year = 2017
num_of_tweets = 300 #max can be 600 for each account but since each tweet doesnt have an image so entire 600 would not be possible

UserTags, TimeStamps, Tweets_url, url = [], [], [], []

for user, pw, month in zip(usernames, passwords, months):
    
    driver = webdriver.Chrome()
    
    if(month>=10):
        driver.get(f'https://twitter.com/search?q=(from%3Anytimes)%20until%3A{year}-{month}-28%20since%3A{year}-{month}-01&src=typed_query&f=live')
    else:
        driver.get(f'https://twitter.com/search?q=(from%3Anytimes)%20until%3A{year}-0{month}-28%20since%3A{year}-0{month}-01&src=typed_query&f=live')


    # Wait for the page to load before continuing
    sleep(7)

    #### LOGIN ######
    # Find the username input field using its XPATH and enter a username
    username = driver.find_element(By.XPATH,"//input[@name='text']")
    username.send_keys(user)

    # Find the 'Next' button using its XPATH and click it to move to the password field
    next_button = driver.find_element(By.XPATH,"//span[contains(text(),'Next')]")
    next_button.click()

    # Wait for the next page to load before continuing
    sleep(10)

    # Find the password input field using its XPATH and enter a password
    password = driver.find_element(By.XPATH,"//input[@name='password']")
    password.send_keys(pw)

    # Find the 'Log in' button using its XPATH and click it to log in
    log_in = driver.find_element(By.XPATH,"//span[contains(text(),'Log in')]")
    log_in.click()


    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    articles = driver.find_elements(By.XPATH,"//article[@data-testid='tweet']")
    
    Tweets2, media_url, Tweets = [], [], []

    begin = time.time()

    while True:
        for article in articles:

            img_url = article.find_elements(By.CLASS_NAME, 'css-9pa8cd')
    #         print(len(img_url))
            if len(img_url) > 1:
                
                try:
                    img_url = img_url[1].get_attribute('src')
                    Tweet = article.find_element(By.XPATH,".//div[@data-testid='tweetText']").text
                    Tweets.append(Tweet)
                    url.append(img_url)
#                     Tweets_url.append((Tweet, img_url))

                    UserTag = article.find_element(By.XPATH,".//div[@data-testid='User-Name']").text
                    UserTags.append(UserTag)

                    TimeStamp = article.find_element(By.XPATH,".//time").get_attribute('datetime')
                    TimeStamps.append(TimeStamp)
                
                    Tweets_url.append((Tweet, img_url, TimeStamp.split('T')[0]))
                    
                except:
                    pass

            else:
                continue

        Tweets2 = list(set(Tweets))
        media_url = list(set(url))

        if len(Tweets2) > num_of_tweets:
            print(len(Tweets2))
            break

        driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
        time.sleep(3)
        articles = driver.find_elements(By.XPATH,"//article[@data-testid='tweet']")

    end = time.time()
    print(f"Scraping done for username: {user}")
    print("Time taken: ", end-begin)
    
    driver.close()

In [32]:
len(Tweets2)

196

In [33]:
tweet, url1, ts = zip(*Tweets_url)
df = pd.DataFrame({"tweetContent": tweet, "mediaUrl": url1, "timeStamp": ts})

In [34]:
df

,tweetContent,mediaUrl
0,Why I Love Reality Television,https://pbs.twimg.com/card_img/171204964837696...
1,"The ancient roads, pyramids and houses reveale...",https://pbs.twimg.com/card_img/171362614936727...
2,As others fled amid a 7.5 magnitude earthquake...,https://pbs.twimg.com/card_img/171256421941632...
3,"""Rarely has the reputation of a leader fallen ...",https://pbs.twimg.com/card_img/171199040042709...
4,Golf Digest and college students helped free a...,https://pbs.twimg.com/card_img/171268104717383...
...,...,...
284,As climate change and rapid growth bring more ...,https://pbs.twimg.com/card_img/171165050324173...
285,Some of the young Chinese hockey players recei...,https://pbs.twimg.com/card_img/171096391985733...
286,"“Hold your burgers, hold your fries. Keep your...",https://pbs.twimg.com/card_img/171085168633439...
287,Two female patients trapped inside a sheriff’s...,https://pbs.twimg.com/card_img/171282150906698...


In [35]:
df.drop_duplicates(inplace=True)
df.index = range(0, len(df))

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 196 entries, 0 to 200
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tweetContent  196 non-null    object
 1   mediaUrl      196 non-null    object
dtypes: object(2)
memory usage: 4.6+ KB


In [38]:
#save to excel
df.to_excel('', index=False)